In [ ]:
# --- Imports and configuration ---

import pandas as pd
import numpy as np
import json
from collections import defaultdict, Counter

# For Google Drive access (only if running in Colab)
from google.colab import drive
drive.mount('/content/drive')

BASE_PATH = '/content/drive/MyDrive/BINOME_WORK/PFE_NIT/AI_CODE_DATASETS/STEP_1_Replay_Atrdf_Modsec/Datasets/'
DATA_PATH = f"{BASE_PATH}/modsec_atrdf_joined.json"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# --- Load ATRDF dataset from JSON ---

with open(DATA_PATH, "r") as f:
    raw_data = json.load(f)
print(f"Loaded {len(raw_data)} entries from JSON file.")

df_raw = pd.DataFrame(raw_data)
print("Columns in raw DataFrame:", df_raw.columns)
print(df_raw.head(2))

Loaded 182767 entries from JSON file.
Columns in raw DataFrame: Index(['request_id', 'primary_key', 'timestamp', 'uri', 'host', 'alerts',
       'original_request'],
      dtype='object')
            request_id primary_key                   timestamp            uri  \
0  174814243710.606864     ATRDF-0  25/May/2025:03:07:17 +0000         /forum   
1  174814243741.357411     ATRDF-1  25/May/2025:03:07:17 +0000  /greet/Capito   

        host                                             alerts  \
0  127.0.0.1  [{'id': '920350', 'msg': 'Host header is a num...   
1  127.0.0.1  [{'id': '920350', 'msg': 'Host header is a num...   

                                    original_request  
0  {'request': {'headers': {'Host': '127.0.0.1:50...  
1  {'request': {'headers': {'Host': '127.0.0.1:50...  


In [ ]:
# --- Flatten original request and response objects ---

def flatten_requests(df):
    def extract_deep_req(x):
        # Cas normal (pas de redondance)
        if isinstance(x, dict) and "request" in x and "response" in x:
            return x
        # Cas redondant (imbrication supplémentaire)
        if isinstance(x, dict) and "original_request" in x and isinstance(x["original_request"], dict):
            x2 = x["original_request"]
            if "request" in x2 and "response" in x2:
                return x2
        return {}

    # Appliquer l'extraction intelligente
    req_extracted = df["original_request"].apply(extract_deep_req)
    # Filtrer pour les entrées valides
    valid_idx = req_extracted.apply(lambda x: isinstance(x, dict) and "request" in x and "response" in x)
    req_valid = req_extracted[valid_idx].reset_index(drop=True)

    # Extraction et flatten comme avant
    request_df = req_valid.apply(lambda x: x["request"]).apply(pd.Series)
    response_df = req_valid.apply(lambda x: x["response"]).apply(pd.Series)
    request_df["headers"] = request_df["headers"].apply(lambda x: x if isinstance(x, dict) else {})
    response_df["headers"] = response_df["headers"].apply(lambda x: x if isinstance(x, dict) else {})
    req_headers = request_df["headers"].apply(pd.Series).add_prefix("req_")
    res_headers = response_df["headers"].apply(pd.Series).add_prefix("res_")
    request_df = request_df.rename(columns={'body': 'req_body'})
    response_df = response_df.rename(columns={'body': 'res_body'})
    df_flat = pd.concat([
        request_df.drop(columns=["headers"]),
        req_headers,
        response_df.drop(columns=["headers"]),
        res_headers
    ], axis=1)
    # Add context columns from the top-level DataFrame (preserving alignment)
    idx_map = df[valid_idx].reset_index(drop=True)
    for col in ['Attack_Tag', 'primary_key', 'uri', 'alerts']:
        if col in idx_map:
            df_flat[col] = idx_map[col].values
    return df_flat


df_flat = flatten_requests(df_raw)

# Keep a copy
df_flat_copy = pd.DataFrame(df_flat)

print(f"Flat DataFrame shape: {df_flat.shape}")
print(df_flat.head(2))

Flat DataFrame shape: (182767, 32)
  method                                                url req_body  \
0    GET  http://127.0.0.1:5000/forum?message=%3C%2Fh2%3...            
1    GET                 http://127.0.0.1:5000/greet/Capito            

  Attack_Tag        req_Host  \
0        XSS  127.0.0.1:5000   
1        NaN  127.0.0.1:5000   

                                      req_User-Agent req_Accept-Encoding  \
0  Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...   gzip, deflate, br   
1  Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKi...   gzip, deflate, br   

  req_Accept req_Connection req_Sec-Ch-Ua-Mobile  ...     status status_code  \
0        */*     keep-alive                   ?0  ...  302 FOUND         302   
1        */*     keep-alive                   ?0  ...     200 OK         200   

                   res_body          res_Content-Type res_Content-Length  \
0  text/html; charset=utf-8  text/html; charset=utf-8                199   
1  text/html; charset=utf-8  

In [ ]:
# --- retrieve the copy ---

df_flat = df_flat_copy.copy()

In [ ]:
# --- Remove noisy/irrelevant alerts ---

def remove_host_header_alerts(alerts):
    """
    Remove 'Host header is a numeric IP address' or id '920350'
    """
    if not isinstance(alerts, list):
        return alerts
    return [
        a for a in alerts
        if a.get("msg") != "Host header is a numeric IP address"
        and a.get("id") != "920350"
    ]

df_flat["alerts"] = df_flat["alerts"].apply(remove_host_header_alerts)

In [ ]:
# --- Define the list of original features to deduplicate on ---

original_features = [
    "method", "url", "req_body", "Attack_Tag",
    "req_Host", "req_User-Agent", "req_Accept-Encoding", "req_Accept",
    "req_Connection", "req_Sec-Ch-Ua-Mobile", "req_Accept-Language",
    "req_Sec-Fetch-Site", "req_Sec-Fetch-Mode", "req_Cache-Control",
    "req_Sec-Fetch-User", "req_Sec-Fetch-Dest", "req_Set-Cookie",
    "req_Date", "req_Sec-Ch-Ua-Platform", "req_Content-Length",
    "req_Cookie", "req_Upgrade-Insecure-Requests", "status",
    "res_body", "res_Content-Type", "res_Content-Length", "res_Location", "res_Set-Cookie"
]

# --- Drop duplicates, keep the first occurrence ---

df_flat_dedup = df_flat.drop_duplicates(subset=original_features, keep='first').reset_index(drop=True)
print(f"Shape after deduplication: {df_flat_dedup.shape}")


Shape after deduplication: (169955, 32)


In [ ]:
# --- Extract the list of rule IDs for each row from the CRS alerts ---

def extract_rule_ids(alerts):
    """
    Returns a list of CRS rule IDs found in the 'alerts' list for each row.
    If no alerts, returns an empty list.
    """
    if not isinstance(alerts, list):
        return []
    return [alert.get("id") for alert in alerts if "id" in alert]

# Apply the function to create a new 'alert_ids' column
df_flat_dedup["alert_ids"] = df_flat_dedup["alerts"].apply(extract_rule_ids)

# --- Flatten the list of all alert IDs across the entire dataset ---

all_alert_ids = [
    aid
    for sublist in df_flat_dedup['alert_ids']
    if isinstance(sublist, list)
    for aid in sublist
]
print(f"Extracted {len(all_alert_ids)} alert IDs in total.")

# --- Count how often each CRS rule ID appears in the dataset ---

id_counts = Counter(all_alert_ids)
print(f"Found {len(id_counts)} unique CRS rule IDs.")

# --- Display most frequent rule IDs in a DataFrame for easy review ---

df_id_counts = (
    pd.DataFrame(list(id_counts.items()), columns=["alert_id", "count"])
    .sort_values("count", ascending=False)
)
print(df_id_counts.head(100))


Extracted 225020 alert IDs in total.
Found 34 unique CRS rule IDs.
   alert_id  count
6    949110  58596
8    932236  24260
9    920340  19291
10   920341  19291
17   942180   7924
14   932240   7253
4    941320   6003
1    941110   6003
3    941390   6003
2    941160   6003
25   942200   4046
7    921151   4037
26   942362   4010
23   942190   4010
27   942380   4010
24   942540   4010
0    941100   4001
5    942131   4001
15   942100   3918
21   942521   3914
20   942520   3914
16   942130   3914
18   942340   3914
19   942370   3582
22   941101   2005
12   944150   1991
13   944151   1991
28   941380   1942
11   920520    575
31   942330    324
30   942390    185
29   942440     94
32   942450      3
33   932235      2


In [ ]:
# --- Map alert rule IDs to severity categories ---

id_to_category = {
    "941100": "severity_xss_libinjection", "941110": "severity_xss_script_tag",
    "941160": "severity_xss_html_injection", "941320": "severity_xss_tag_handler",
    "941390": "severity_xss_javascript_method", "941101": "severity_xss_libinjection_alt",
    "942100": "severity_sqli_libinjection", "942131": "severity_sqli_boolean_based",
    "942190": "severity_sqli_injection_attack", "942200": "severity_sqli_union_select",
    "942210": "severity_sqli_boolean_based_alt", "942180": "severity_sqli_auth_bypass",
    "942540": "severity_sqli_concat_basic", "942520": "severity_sqli_auth_bypass_4_0",
    "942521": "severity_sqli_auth_bypass_4_1", "942130": "severity_sqli_boolean_based_alt2",
    "932236": "severity_rce_unix_command_injection", "932240": "severity_rce_unix_command_evasion",
    "932200": "severity_rce_java", "932210": "severity_rce_log4j", "932160": "severity_rce_unix_shell",
    "932170": "severity_rce_python", "932230": "severity_rce_cmd_injection",
    "920272": "severity_log_forging_crlf", "920300": "severity_cookie_malformed",
    "921151": "severity_http_header_injection", "920340": "severity_content_no_type",
    "920341": "severity_content_requires_type", "949110": "severity_anomaly_score_exceeded",
    "920350": "severity_protocol_numeric_host", "920520": "severity_accept_encoding_too_long",
    "941380": "severity_xss_angularjs_template", "944150": "severity_java_serialized_object",
    "944151": "severity_java_serialized_object_alt"
}

# Map alert_severity_map for each row

alert_severity_map = {}
for idx, alerts in enumerate(df_flat_dedup['alerts']):
    if isinstance(alerts, list):
        for alert in alerts:
            aid = alert.get('id')
            sev = alert.get('severity')
            if aid and sev:
                alert_severity_map[(idx, aid)] = int(sev)

from collections import defaultdict
import pandas as pd

def categorize_alerts(alert_ids, row_idx):
    categories = defaultdict(int)
    for aid in alert_ids:
        category = id_to_category.get(aid)
        if category:
            sev = alert_severity_map.get((row_idx, aid), 0)
            categories[category] += sev
    return pd.Series(categories)

df_flat_dedup = df_flat_dedup.join(
    df_flat_dedup.apply(lambda row: categorize_alerts(row["alert_ids"], row.name), axis=1)
)


In [ ]:
# --- Binarize all severity columns: 0 or 1 ---

severity_cols = [col for col in df_flat_dedup.columns if col.startswith('severity_')]
df_flat_dedup[severity_cols] = df_flat_dedup[severity_cols].fillna(0)
df_flat_dedup[severity_cols] = (df_flat_dedup[severity_cols] > 0).astype(int)

# --- Recompute binary alert count ---

df_flat_dedup['alert_count'] = df_flat_dedup[severity_cols].sum(axis=1)


In [ ]:
# --- Drop useless columns for ML pipeline ---

cols_to_drop = ['alerts', 'primary_key', 'uri', 'status_code', 'alert_ids']
for col in cols_to_drop:
    if col in df_flat_dedup:
        df_flat_dedup = df_flat_dedup.drop(columns=[col])

# --- Save final cleaned & flattened dataset ---

OUTPUT_PATH = '/content/drive/MyDrive/BINOME_WORK/PFE_NIT/AI_CODE_DATASETS/STEP_2_AI_Training/Datasets/df_flat.csv'
df_flat_dedup.to_csv(OUTPUT_PATH, index=False)
print(f"Final cleaned DataFrame saved: {OUTPUT_PATH}")
print("Final DataFrame shape:", df_flat_dedup.shape)


Final cleaned DataFrame saved: /content/drive/MyDrive/BINOME_WORK/PFE_NIT/AI_CODE_DATASETS/STEP_2_AI_Training/Datasets/df_flat.csv
Final DataFrame shape: (169955, 54)


In [ ]:
# --- Quick stats for reporting ---

print(df_flat_dedup.head())
print("Unique Attack_Tags:", df_flat_dedup['Attack_Tag'].unique())
print(df_flat_dedup['alert_count'].value_counts())
print("Feature columns:", df_flat_dedup.columns.tolist())


  method                                                url  \
0    GET  http://127.0.0.1:5000/forum?message=%3C%2Fh2%3...   
1    GET                 http://127.0.0.1:5000/greet/Capito   
2    GET                    http://127.0.0.1:5000/bookstore   
3    GET  http://127.0.0.1:5000/orders/check/exists?val=...   
4   POST  http://127.0.0.1:5000/categories/check/all?id=...   

               req_body   Attack_Tag        req_Host  \
0                                XSS  127.0.0.1:5000   
1                                NaN  127.0.0.1:5000   
2                                NaN  127.0.0.1:5000   
3                        Log Forging  127.0.0.1:5000   
4  MkdXwyHaWCdnLhoXJDzY          NaN  127.0.0.1:5000   

                                      req_User-Agent  \
0  Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKi...   
1  Mozilla/5.0 (Windows NT 4.0; WOW64) AppleWebKi...   
2                             python-requests/2.28.1   
3  Mozilla/5.0 (X11; Linux x86_64; rv:28.0) Gecko...   
4  M